## <a href="https://cursos.alura.com.br/course/langchain-python-ferramentas-llm-openai/task/156164"><b>Formatando a saída de uma cadeia com o JSONOutputParser</b></a><br/>
Dar restart no kernel para limpar a memória. Não esquecer de liberar o scroll na última célula, para ver a saída completa.

O objetivo é obter apenas a informação que desejo, e nenhum "lixo" a mais da IA

In [64]:
from langchain.prompts import ChatPromptTemplate # Uma interação de múltiplos prompts é um chat, por isso, vamos importar um ChatPromptTemplate

In [65]:
%pip install -qr requirements.txt

Note: you may need to restart the kernel to use updated packages.


#### <b>PASSO 1 - IMPORTS e CRIAÇÃO DA LLM</b>

In [66]:
from langchain_openai import ChatOpenAI
from os import getenv
from dotenv import load_dotenv # CARREGA A VARIÁVEL DE AMBIENTE OPENAI_KEY LIDA DO ARQUIVO .env

load_dotenv() # CARREGANDO O ARQUIVO COM A OPENAI_KEY

llm = ChatOpenAI( # INSTANCIANDO A LLM
                    model="gpt-4.1-mini",
                    temperature=0.5,
                    # 1 - OBTENDO A API KEY POR MEIO DA VARIÁVEL DE AMBIENTE OPENAI_KEY. QUE VAI FICAR ARMAZENADA NO ARQUIVO .env.
                    # 2 - AINDA É NECESSÁRIO CARREGAR ESSE ARQUIVO. VER NA PRIMEIRA CÉLULA DO NOTEBOOK
                    api_key=getenv("OPENAI_KEY")
                )

#### <b>PASSO 2 - CRIANDO O <i>PROMPT TEMPLATE</i> E ASSOCIANDO UM PARSER A ELE</b></br> 

<b><ol><li>CRIANDO OS MODELOS</li></ol></b>

<ul><ul><li><b>CRIANDO O MODELO PARA A CIDADE</b></li></ul></ul>

In [67]:
# ANTES
template_cidade = ChatPromptTemplate.from_template("Sugira uma cidade, dado o meu interesse por {interesse}") # INSTANCIANDO ChatPromptTemplate e INICIANDO A PARTIR DE UM TEMPLATE

<ul><ul><li><b>CRIANDO O PARSER E ASSOCIANDO ELE AO MODELO DE CIDADE</b></li></ul></ul>

<ul><ul><ul><li><b>Quero obter apenas o nome da cidade e o motivo, nenhuma outra informação a mais. Necessário trabalhar a saida.</b></li></ul></ul></ul>

<ul><ul><ul><ul><li><b>PASSO 1 - Criar uma classe minha com os campos que preciso. Descreve o formato da saída</b></li></ul></ul></ul></ul>

In [68]:
from pydantic import Field,BaseModel # pydantic -> Biblioteca para validação de dados. Garante que os dados recebidos ou manipulados estejam no formato correto,
                                     # BaseModel -> Os modelos pydantic são classes que herdam BaseModel (https://docs-pydantic-dev.translate.goog/latest/concepts/models/?_x_tr_sl=en&_x_tr_tl=pt&_x_tr_hl=pt&_x_tr_pto=tc)
                                     # Modelos possuem campos como atributos.
                                     
                                     # Field -> Para personalizar os campos do modelo (https://docs-pydantic-dev.translate.goog/latest/concepts/fields/?_x_tr_sl=en&_x_tr_tl=pt&_x_tr_hl=pt&_x_tr_pto=tc)

class Destino(BaseModel): # A nossa classe vai estender a classe BaseModel, que terá dois campos a cidade e o motivo de visitá-la
    cidade: str = Field(description="cidade a visitar") # Descrição do campo. Apenas informativo
    motivo: str = Field(description="motivo pelo qual é interessante visitar") # Descrição do campo. Apenas informativo  

<ul><ul><ul><ul><li><b>PASSO 2 - Criando o parser e associando ele ao template. Dessa vez um PromptTemplate e não um ChatPromptTemplate</b></li></ul></ul></ul></ul>
<ul><ul><ul><ul><ul><li>VARÍAVEL PARCIAL É O SHOT DA ENGENHARIA DE PROMPT</li></ul></ul></ul></ul></ul>



In [69]:
# DEPOIS
from langchain import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser # EXISTEM DIVERSOS OUTPUT PARSERS (https://python.langchain.com/docs/concepts/output_parsers/)

parseador = JsonOutputParser(pydantic_object=Destino) # DOCUMENTAÇÃO JsonOutputParser (https://python.langchain.com/docs/how_to/output_parser_json)

template_cidade = PromptTemplate(
                                    template="""Sugira uma cidade, dado o meu interesse por {interesse}.
                                    {formatacao_de_saida_da_ia}""", # AQUI TEMOS UMA VARÍAVEL PARCIAL. UTLIZAÇÃO DA TÉCNICA DE SHOTS PARA PROMPTS
                                    input_variables=["interesse"],
                                    # A VARÍAVEL PARCIAL É UM DICIONÁRIO. FUNCIONA COMO O SHOT
                                    partial_variables={"formatacao_de_saida_da_ia":parseador.get_format_instructions()}, # PASSANDO O PARSEADOR PARA A VARIÁVEL FORMATAÇÃO DE SAÍDA.
                                                                                                   
                                ) # INSTANCIANDO PromptTemplate e INICIANDO A PARTIR DE UM TEMPLATE


<b>DETALHES SOBRE A DOCUMENTAÇÃO</b></br></br>
Ele coloca a tipagem do Field como string (opcional no nosso caso) e faz outras coisas. Ao rolar a página para baixo, veremos que existem outros OutputParsers de saída, como os de XML, de CSV, de data, que retornam uma opção entre 5, que retornam um output estruturado em chave-valor, e assim por diante.

Também podemos criar nosso próprio OutputParser, que vai dar as instruões e devolver no formato que quisermos utilizar. Contudo, não há garantia que ele vai funcionar sempre, assim como ele não garante nada na saída.

<ul><ul><li><b>CRIANDO O MODELO PARA RESTAURANTES</b></li></ul></ul>

In [70]:
template_restaurante = ChatPromptTemplate.from_template("Sugira restaurantes populares entre locais na {cidade}") # INSTANCIANDO ChatPromptTemplate e INICIANDO A PARTIR DE UM TEMPLATE

<ul><ul><li><b>CRIANDO O MODELO CULTURAL</b></li></ul></ul>

In [71]:
template_cultural = ChatPromptTemplate.from_template("Sugira atividades e locais culturais em {cidade}") # INSTANCIANDO ChatPromptTemplate e INICIANDO A PARTIR DE UM TEMPLATE

#### <b>PASSO 3 - CRIANDO AS CADEIAS</b></br> 

In [72]:
from langchain.chains import LLMChain

cadeia_cidade = LLMChain(llm=llm,prompt=template_cidade)
cadeia_restaurante = LLMChain(llm=llm,prompt=template_restaurante)
cadeia_cultural = LLMChain(llm=llm,prompt=template_cultural)

<b><ul><li>CRIANDO A CADEIA GERAL (SimpleSequentialChain)</li></ul></b>

In [73]:
from langchain.chains import SimpleSequentialChain

cadeia = SimpleSequentialChain(chains=[cadeia_cidade,cadeia_restaurante,cadeia_cultural])


#### <b>PASSO 4 - INVOCANDO A CADEIA GERAL</b>

In [76]:
resposta = cadeia.invoke("praias") # DIFERENTE DO QUE ACONTECEU COM O PROMPT TEMPLATE, AQUI INVOCAMOS A CADEIA, E COMTÉM UM TEMPLATE, E ELA INVOCA A LLM
print(resposta)

[chain/start] [chain:SimpleSequentialChain] Entering Chain run with input:
{
  "input": "praias"
}
[chain/start] [chain:SimpleSequentialChain > chain:LLMChain] Entering Chain run with input:
{
  "interesse": "praias"
}
[llm/start] [chain:SimpleSequentialChain > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Sugira uma cidade, dado o meu interesse por praias.\n                                    The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {\"properties\": {\"foo\": {\"title\": \"Foo\", \"description\": \"a list of strings\", \"type\": \"array\", \"items\": {\"type\": \"string\"}}}, \"required\": [\"foo\"]}\nthe object {\"foo\": [\"bar\", \"baz\"]} is a well-formatted instance of the schema. The object {\"properties\": {\"foo\": [\"bar\", \"baz\"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{\"properties\": {\"cidade\": {\"description\": \"cidade a v

#### <b>OBTENDO APENAS O CONTEÚDO DA MENSAGEM</b>

In [ ]:
print(resposta['output'])

<ul><li><b>USANDO O MODO DEBUG</b></li></ul>

In [ ]:
from langchain.globals import set_debug
set_debug(True)

resposta = cadeia.invoke(input="praias")
print(resposta)

<b>RESULTADO DOS PROMPTS</b>

"prompts": [
    "Human: Sugira uma cidade, dado o meu interesse por praias.\n                                    The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {\"properties\": {\"foo\": {\"title\": \"Foo\", \"description\": \"a list of strings\", \"type\": \"array\", \"items\": {\"type\": \"string\"}}}, \"required\": [\"foo\"]}\nthe object {\"foo\": [\"bar\", \"baz\"]} is a well-formatted instance of the schema. The object {\"properties\": {\"foo\": [\"bar\", \"baz\"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{\"properties\": {\"cidade\": {\"description\": \"cidade a visitar\", \"title\": \"Cidade\", \"type\": \"string\"}, \"motivo\": {\"description\": \"motivo pelo qual é interessante visitar\", \"title\": \"Motivo\", \"type\": \"string\"}}, \"required\": [\"cidade\", \"motivo\"]}\n```"
  ]

[<font style="color: orange">llm/start</font>] [chain:SimpleSequentialChain > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:</br>
{</br>
  <ul>"prompts": [</br>
    <ul><ul>"Human: Sugira restaurantes populares entre locais na ```json\n{\n  \"cidade\": \"Florianópolis\",\n  \"motivo\": \"Possui diversas praias paradisíacas, com águas claras e ótima infraestrutura para turistas.\"\n}\n```"</ul></ul>
  ]</ul>
}</br>
[<font style="color: orange">llm/end</font>] [chain:SimpleSequentialChain > chain:LLMChain > llm:ChatOpenAI] [5.86s] Exiting LLM run with output:


"prompts": [
    "Human: Sugira atividades e locais culturais em ```json\n{\n  \"cidade\": \"Florianópolis\",\n  \"motivo\": \"Possui diversas praias paradisíacas, com águas claras e ótima infraestrutura para turistas.\",\n  \"restaurantes_populares_entre_locais\": [\n    {\n      \"nome\": \"Ostradamus\",\n      \"descricao\": \"Restaurante tradicional especializado em frutos do mar, famoso pela qualidade das ostras e pratos frescos.\",\n      \"localizacao\": \"Praia de Ribeirão da Ilha\"\n    },\n    {\n      \"nome\": \"Bar do Boni\",\n      \"descricao\": \"Bar e restaurante com ambiente descontraído, conhecido pelos petiscos e pratos típicos catarinenses.\",\n      \"localizacao\": \"Centro de Florianópolis\"\n    },\n    {\n      \"nome\": \"Box 32\",\n      \"descricao\": \"Restaurante na Beira-Mar Norte, muito apreciado por moradores locais pela variedade de frutos do mar e ambiente agradável.\",\n      \"localizacao\": \"Beira-Mar Norte\"\n    },\n    {\n      \"nome\": \"Restaurante Ponta das Caranhas\",\n      \"descricao\": \"Especializado em frutos do mar, com vista privilegiada para o mar e ambiente rústico chique.\",\n      \"localizacao\": \"Ponta das Caranhas\"\n    },\n    {\n      \"nome\": \"Cantina Sangiovese\",\n      \"descricao\": \"Cantina italiana tradicional, muito frequentada por locais que apreciam massas artesanais e vinhos selecionados.\",\n      \"localizacao\": \"Lagoa da Conceição\"\n    }\n  ]\n}\n```"
  ]

#### <b>DatetimeOutputParser</b></br>
O DatetimeOutputParser é outro exemplo útil para transformar texto em dados de data e hora. No entanto, desafios podem surgir se o formato de data e hora esperado não for diretamente suportado pelo parser.

In [ ]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from os import getenv
from dotenv import load_dotenv # CARREGA A VARIÁVEL DE AMBIENTE OPENAI_KEY LIDA DO ARQUIVO .env

from langchain.globals import set_debug
set_debug(True)

load_dotenv()

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.5,
    api_key=getenv("OPENAI_KEY"))

parseador_saida = DatetimeOutputParser()
modelo_data = """Responda a pergunta do usuário: 
    {pergunta}

    {formato_saida}
"""

prompt = PromptTemplate.from_template(
    template=modelo_data,
    partial_variables={"formato_saida": parseador_saida.get_format_instructions()}
)

# SOBREPONDO OS VALORES ACIMA
PromptTemplate(
    input_variables=['question'],
    partial_variables={'formato_saida': """Write a datetime string that matches the following pattern: 
                       '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 0668-08-09T12:56:32.732651Z, 1213-06-23T21:01:36.868629Z, 0713-07-06T18:19:02.257488Z
                       \n\nReturn ONLY this string, no other words!"""}, 
    template='Answer the users question:\n\n{question}\n\n{format_instructions}'
)

# AQUI NÃO FOI USADO SimpleSequentialChain
chain = prompt | llm | parseador_saida

resposta = chain.invoke(input={"pergunta": "Quando a bitcoin foi fundada?"}) # AQUI USOU UM DICIONÁRIO, PORQUE O TEMPLATE TEM DOIS PLACEHOLDERS {pergunta} e {formato_saida}

print(resposta)


#### <b>JsonOutputParser e PydanticOutputParser</b></br>
O JsonOutputParser é particularmente útil quando a saída necessita ser mapeada em diferentes categorias ou itens. Suporta classes Pydantic, facilitando a transformação da saída do LLM em objetos estruturados e prontos para uso em aplicações. Isso é extremamente útil para sumarizar dados complexos, como tickets de suporte, em categorias distintas como Issue, Root Causes e Resolution.

Exemplo

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from os import getenv
from dotenv import load_dotenv # CARREGA A VARIÁVEL DE AMBIENTE OPENAI_KEY LIDA DO ARQUIVO .env

load_dotenv()

from langchain.globals import set_debug
set_debug(True)


llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.5,
    api_key=getenv("OPENAI_KEY"))

# Defina a classe com a estrutura desejada
class Bandeira(BaseModel):
    pais: str = Field(description="nome do pais")
    cores: str = Field(description="cor principal da bandeira")
    historia: str = Field(description="história da bandeira")

# Defina o prompt que será utilizado para pergunta
flag_query = "Me fale da bandeira do Brasil"

# Defina a estrutura que será utilizada para processar a saída
parseador_bandeira = JsonOutputParser(pydantic_object=Bandeira)

prompt = PromptTemplate(
    template="Responda a pergunta do usuário.\n{pergunta}\n{instrucoes_formato}\n", # PROMPT TEMPLATE COM A PERGUNTA E SHOT DE SAÍDA
    input_variables=["pergunta"],
    partial_variables={"instrucoes_formato": parseador_bandeira.get_format_instructions()},
)

chain = prompt | llm | parseador_bandeira

chain.invoke({"pergunta": flag_query})